# Verify device with Torch

In [1]:
import torch

In [2]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('Number of CUDA Devices:', torch.cuda.device_count())
    print('CUDA Device Name:',torch.cuda.get_device_name(0))
    print('CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
    print('CUDA version:', torch.version.cuda)
else:
    print('No CUDA-GPU available from Torch perspective.')

No CUDA-GPU available from Torch perspective.


In [3]:
torch.cuda.is_available()

False

In [4]:
torch.cuda.device_count()

0

In [5]:
torch.cuda.current_device()

AssertionError: Torch not compiled with CUDA enabled

# Verify device with Tensorflow-gpu

In [6]:
import tensorflow as tf
from tensorflow.python.client import device_lib

/home/c3/.conda/envs/py-copycat/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/c3/.conda/envs/py-copycat/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/c3/.conda/envs/py-copycat/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c3/.conda/envs/py-copycat/lib/python3.6/site-pack

In [7]:
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4692077885660092708
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11338085172
locality {
  bus_id: 1
}
incarnation: 208304305281607498
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0001:00:00.0"
]


In [8]:
# is it really a cuda device?
tf.test.is_built_with_cuda()

True

# Matrix operations using Tensorflow-gpu

In [9]:
import numpy as np
import tensorflow as tf
from time import perf_counter

### Define matrix multiplication function with TF

In [10]:
def matpow(M, n):
    """
    Takes in matrix M and multiplies it by itself n-1 times.
    """
    if n < 1:
        return M
    else:
        return tf.matmul(M, matpow(M,n-1))

### Define matrices A and B, n to calculate (A^n + B^n )

In [11]:
size = 10000
A = np.random.rand(size, size).astype('float32')
B = np.random.rand(size, size).astype('float32')
n = 10

## GPU multiplication

This is pretty much all taken from: https://gist.github.com/j-min/baae1aa56e861cab9831b3722755ae6d

In [12]:
c1 = []
c2 = []

with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [size, size])
    b = tf.placeholder(tf.float32, [size, size])
    c1.append(matpow(a, n))
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c1)

start_time = perf_counter()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
stop_time = perf_counter()

print('GPU time: %g s' % (stop_time-start_time))

GPU time: 8.47029 s


## CPU multiplication

In [13]:
c1 = []
c2 = []

with tf.device('/cpu:0'):
    a = tf.placeholder(tf.float32, [size, size])
    b = tf.placeholder(tf.float32, [size, size])
    c1.append(matpow(a, n))
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c1)

start_time = perf_counter()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
stop_time = perf_counter()

print('CPU time: %g s' % (stop_time-start_time))

CPU time: 857.143 s


# Numba vectorization in the GPU -- WIP

In [14]:
import numpy as np
from time import perf_counter
from numba import vectorize

ModuleNotFoundError: No module named 'numba'

In [ ]:
@vectorize(['float32(float32, float32)'], target='cuda')
def add_vec(v1, v2):
    return v1 + v2



N=1<<12

a = np.ones(N, dtype=np.float32)
b = np.ones(N, dtype=np.float32)
c = np.empty_like(a, dtype=a.dtype)

start_time = perf_counter()
c = add_vec(a,b)
stop_time = perf_counter()

print(c)
print('Elapsed time: %g s' % (stop_time-start_time))

del a
del b
del c

In [ ]:
del a
del b
del c

In [ ]:
# Testing the version of packages #
tf.__version__

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))